## Primeira versão Auto-pytorch - entendimento da biblioteca
- https://github.com/automl/Auto-PyTorch
- https://www.automl.org/automl-for-x/tabular-data/autopytorch/
- https://automl.github.io/Auto-PyTorch/development/
- https://automl.github.io/Auto-PyTorch/development/manual.html

# Esse notebook foi rodado dentro do .devcontainer para a imagem automlorg/autopytorch:v0.2 que foi pega do docker hub

## Comandos necessarios para rodar o AutoPytorch
Esses comandos já foram adicionados no dev container

In [ ]:
#  Precisei instalar isso para conseguir rodar a TabularClassificationTask
! apt-get update

In [ ]:
!apt-get install libgl1-mesa-glx -y

In [19]:
! apt-get install libglib2.0-0 -y
# dpkg --configure -a

In [ ]:
# Opcional
# apt-get install htop

## Tentativa 1

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('../../auto-pytorch') 

In [3]:
from autoPyTorch.api.tabular_classification import TabularClassificationTask

In [4]:
# data and metric imports
import sklearn.model_selection
import sklearn.datasets
import sklearn.metrics


In [5]:
X, y = sklearn.datasets.load_digits(return_X_y=True)
X_train, X_test, y_train, y_test = \
        sklearn.model_selection.train_test_split(X, y, random_state=1)

In [6]:
# initialise Auto-PyTorch api
api = TabularClassificationTask()

In [7]:
# Search for an ensemble of machine learning algorithms
api.search(
    X_train=X_train,
    y_train=y_train,
    X_test=X_test,
    y_test=y_test,
    optimize_metric='accuracy',
    total_walltime_limit=300,
    func_eval_time_limit_secs=50
)

cannot allocate memory for thread-local data: ABORT
cannot allocate memory for thread-local data: ABORT


[ERROR] [2025-01-26 15:58:12,878:Client-AutoPyTorch:3e94edeb-dc28-11ef-a2a1-0242ac110002:1] Traditional prediction for lgb failed with run state StatusType.CRASHED.
Additional info:
error: Result queue is empty
exit_status: <class 'pynisher.limit_function_call.AnythingException'>
subprocess_stdout: 
subprocess_stderr: 
exitcode: 127
configuration_origin: traditional
[ERROR] [2025-01-26 15:58:42,765:Client-AutoPyTorch:3e94edeb-dc28-11ef-a2a1-0242ac110002:1] Traditional prediction for catboost failed with run state StatusType.CRASHED.
Additional info:
error: Result queue is empty
exit_status: <class 'pynisher.limit_function_call.AnythingException'>
subprocess_stdout: 
subprocess_stderr: 
exitcode: -11
configuration_origin: traditional


OpenBLAS blas_thread_init: pthread_create failed for thread 16 of 32: Resource temporarily unavailable
OpenBLAS blas_thread_init: RLIMIT_NPROC -1 current, -1 max
OpenBLAS blas_thread_init: pthread_create failed for thread 17 of 32: Resource temporarily unavailable
OpenBLAS blas_thread_init: RLIMIT_NPROC -1 current, -1 max
OpenBLAS blas_thread_init: pthread_create failed for thread 18 of 32: Resource temporarily unavailable
OpenBLAS blas_thread_init: RLIMIT_NPROC -1 current, -1 max
OpenBLAS blas_thread_init: pthread_create failed for thread 19 of 32: Resource temporarily unavailable
OpenBLAS blas_thread_init: RLIMIT_NPROC -1 current, -1 max
OpenBLAS blas_thread_init: pthread_create failed for thread 20 of 32: Resource temporarily unavailable
OpenBLAS blas_thread_init: RLIMIT_NPROC -1 current, -1 max
OpenBLAS blas_thread_init: pthread_create failed for thread 21 of 32: Resource temporarily unavailable
OpenBLAS blas_thread_init: RLIMIT_NPROC -1 current, -1 max
OpenBLAS blas_thread_init: p

[ERROR] [2025-01-26 15:59:43,200:Client-AutoPyTorch:3e94edeb-dc28-11ef-a2a1-0242ac110002:1] Traditional prediction for knn failed with run state StatusType.TIMEOUT.
Additional info:
error: Timeout
configuration_origin: traditional


/usr/local/lib/python3.8/dist-packages/smac/intensification/parallel_scheduling.py:154: UserWarning: Hyperband is executed with 1 workers only. Consider to use pynisher to use all available workers.
  warnings.warn(
cannot allocate memory for thread-local data: ABORT
cannot allocate memory for thread-local data: ABORT
cannot allocate memory for thread-local data: ABORT
cannot allocate memory for thread-local data: ABORT
cannot allocate memory for thread-local data: ABORT
OpenBLAS blas_thread_init: pthread_create failed for thread 16 of 32: Resource temporarily unavailable
OpenBLAS blas_thread_init: RLIMIT_NPROC -1 current, -1 max
OpenBLAS blas_thread_init: pthread_create failed for thread 17 of 32: Resource temporarily unavailable
OpenBLAS blas_thread_init: RLIMIT_NPROC -1 current, -1 max
OpenBLAS blas_thread_init: pthread_create failed for thread 18 of 32: Resource temporarily unavailable
OpenBLAS blas_thread_init: RLIMIT_NPROC -1 current, -1 max
OpenBLAS blas_thread_init: pthread_cre

In [8]:
# Calculate test accuracy
y_pred = api.predict(X_test)
score = api.score(y_pred, y_test)
print("Accuracy score", score)

Accuracy score {'accuracy': 0.9844444444444445}


In [9]:
print(api.show_models())

|    | Preprocessing   | Estimator   |   Weight |
|---:|:----------------|:------------|---------:|
|  0 | None            | SVMLearner  |     0.96 |
|  1 | None            | RFLearner   |     0.04 |


In [10]:
print(api.sprint_statistics())

autoPyTorch results:
	Dataset name: 3e94edeb-dc28-11ef-a2a1-0242ac110002
	Optimisation Metric: accuracy
	Best validation score: 0.9842696629213483
	Number of target algorithm runs: 22
	Number of successful target algorithm runs: 3
	Number of crashed target algorithm runs: 13
	Number of target algorithms that exceeded the time limit: 5
	Number of target algorithms that exceeded the memory limit: 1



In [ ]:
help(api)

## Tentativa 2

In [3]:
import os
import tempfile as tmp
import warnings

from autoPyTorch.datasets.resampling_strategy import CrossValTypes

os.environ['JOBLIB_TEMP_FOLDER'] = tmp.gettempdir()
os.environ['OMP_NUM_THREADS'] = '1'
os.environ['OPENBLAS_NUM_THREADS'] = '1'
os.environ['MKL_NUM_THREADS'] = '1'

warnings.simplefilter(action='ignore', category=UserWarning)
warnings.simplefilter(action='ignore', category=FutureWarning)

import sklearn.datasets
import sklearn.model_selection

from autoPyTorch.api.tabular_classification import TabularClassificationTask

In [4]:
import pandas as pd
from glob import glob

In [5]:
pd.set_option('display.max_columns', None)

In [6]:
csv_files = glob("../Data/data_Q4_2023/*/*csv", recursive=True)
# df = pd.concat([pd.read_csv(file) for file in csv_files], ignore_index=True)

In [7]:
columns_to_drop = ["date", "datacenter", "cluster_id", "vault_id",	"pod_id"]
columns_to_set = ["serial_number"]

In [10]:
df = pd.read_csv(csv_files[0])
df = df.drop(columns=columns_to_drop)

In [11]:
index_to_set_falure = list(range(10_000)) 
df.loc[index_to_set_falure, "failure"] = 1

df = df.set_index(columns_to_set)


In [12]:
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(
    df.drop(columns=["failure"]),
    df[["failure"]],
    random_state=12_01_2001,
    shuffle=True,
    stratify=df["failure"],
    test_size=0.2
)

In [13]:
api = TabularClassificationTask(
    # To maintain logs of the run, you can uncomment the
    # Following lines
    # temporary_directory='./tmp/autoPyTorch_example_tmp_01',
    # output_directory='./tmp/autoPyTorch_example_out_01',
    # delete_tmp_folder_after_terminate=False,
    # delete_output_folder_after_terminate=False,
    seed=42,
)

In [ ]:
type(y_train)#.toframe()

In [ ]:

X_train.head()

In [96]:
feat_types = []

In [ ]:
api.search(
    X_train=X_train,
    y_train=y_train,
    X_test=X_test.copy(),
    y_test=y_test.copy(),
    dataset_name='Trabalho_deep_learning',
    optimize_metric='accuracy',
    total_walltime_limit=300,
    func_eval_time_limit_secs=50
)
# node.get_hyperparameter_search_space(dataset_properties, include=choices_list)

In [15]:
# help(TabularClassificationTask)